In [ ]:
import numpy as np
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from framework.mtl_model import MTLModel
from framework.trainer import Trainer
from data.dataloader.cityscapes_dataloader import CityScapes
from data.heads.pixel2pixel import ASPPHeadNode
from data.metrics.pixel2pixel_loss import CityScapesCriterions
from data.metrics.pixel2pixel_metrics import CityScapesMetrics

# Data

In [ ]:
dataroot = 'datasets/Cityscapes/'

headsDict = nn.ModuleDict()
trainDataloaderDict = {}
valDataloaderDict = {}
criterionDict = {}
metricDict = {}

tasks = ['segment_semantic', 'depth_zbuffer']
task_cls_num = {'segment_semantic': 19, 'depth_zbuffer': 1}
for task in tasks:
    headsDict[task] = ASPPHeadNode(512, task_cls_num[task])

    # For model trainer
    dataset = CityScapes(dataroot, 'train', task, crop_h=224, crop_w=224)
    trainDataloaderDict[task] = DataLoader(dataset, 16, shuffle=True)

    dataset = CityScapes(dataroot, 'test', task)
    valDataloaderDict[task] = DataLoader(dataset, 16, shuffle=True)

    criterionDict[task] = CityScapesCriterions(task)
    metricDict[task] = CityScapesMetrics(task)

# Model

In [ ]:
prototxt = 'models/deeplab_resnet34_adashare.prototxt'
# prototxt = 'models/mobilenetv2.prototxt' # the input dim of heads should be changed to 1280
# prototxt = 'models/mnasnet.prototxt' # the input dim of heads should be changed to 1280
mtlmodel = MTLModel(prototxt, headsDict)
mtlmodel = mtlmodel.cuda()

# Train

In [ ]:
checkpoint = 'checkpoint/'
trainer = Trainer(mtlmodel, trainDataloaderDict, valDataloaderDict, criterionDict, metricDict, 
                  print_iters=100, val_iters=500, save_num=1, policy_update_iters=100)

In [ ]:
trainer.pre_train(iters=10000, lr=0.0001, savePath=checkpoint+'Cityscapes/')

In [ ]:
loss_lambda = {'segment_semantic': 1, 'depth_zbuffer': 1, 'policy':0.0005}
trainer.alter_train_with_reg(iters=20000, policy_network_iters=(100,400), policy_lr=0.01, network_lr=0.0001, 
                             loss_lambda=loss_lambda,
                             savePath=checkpoint+'Cityscapes/')

In [ ]:
loss_lambda = {'segment_semantic': 1, 'depth_zbuffer': 1}
trainer.post_train(iters=40000, lr=0.001, 
                   decay_lr_freq=4000, decay_lr_rate=0.5,
                   loss_lambda=loss_lambda,
                   savePath=checkpoint+'Cityscapes/', reload='alter_train_with_reg0005.model')